In [5]:
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
from pybaseball import cache

cache.enable()


Gathering player lookup table. This may take a moment.
Gathering Player Data
Gathering Player Data
   pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
0          ST  2014-08-16           85.3          -2.04           5.63   
1          FF  2014-08-16           94.8          -2.06           5.70   
2          FF  2014-08-16           93.4          -2.13           5.72   
3          FF  2014-08-16           93.6          -2.17           5.60   
4          KC  2014-08-16           78.6          -2.42           5.64   
5          FF  2014-08-16           92.6          -2.03           5.69   
6          FF  2014-08-16           93.3          -2.29           5.70   
7          FF  2014-08-16           94.0          -2.16           5.59   
8          CH  2014-08-16           79.6          -2.21           5.90   
9          FF  2014-08-16           93.3          -2.03           5.56   
10         FF  2014-08-16           94.0          -2.17           5.59   
11         KC

In [ ]:
# find David Ortiz's player id (mlbam_key)
playerid_lookup('ortiz','david')

# get all available data
data = statcast_batter('2008-04-01', '2017-07-15', player_id = 120074)

# get data for August 16th, 2014
data = statcast_batter('2014-08-16', player_id = 120074)

print(data)

In [8]:
# Sample list of player names
player_names = ['david ortiz', 'mike trout', 'bryce harper']  # Add more player names as needed

# Initialize an empty DataFrame to store the OPS data
ops_data = pd.DataFrame(columns=['Player', 'Year', 'OPS'])

# Loop through each player name
for player_name in player_names:
    # Lookup player ID using playerid_lookup
    player_info = playerid_lookup(player_name.split()[1], player_name.split()[0])
    
    if not player_info.empty:
        player_id = player_info['key_mlbam'].iloc[0]
        
        # Fetch player data for the desired date range
        player_data = statcast_batter(start_dt='2008-04-01', end_dt='2023-08-13', player_id=player_id)
        
        player_data['game_date'] = pd.to_datetime(player_data['game_date'])
        # Calculate OPS for each year
        yearly_ops = player_data.groupby(player_data['game_date'].dt.year)['woba_value'].mean()
        
        # Append OPS data to the DataFrame
        ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),
                                                 'Year': yearly_ops.index,
                                                 'OPS': yearly_ops}), ignore_index=True)
    else:
        print(f"Player {player_name} not found.")

# Display the collected OPS data
ops_data

Gathering Player Data


/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_34312/1802613251.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),


Gathering Player Data


/Applications/anaconda3/lib/python3.9/site-packages/pybaseball/utils.py:295: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))
/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_34312/1802613251.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),


Gathering Player Data


/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_34312/1802613251.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),


,Player,Year,OPS
0,david ortiz,2008,0.361589
1,david ortiz,2009,0.342547
2,david ortiz,2010,0.363279
3,david ortiz,2011,0.392039
4,david ortiz,2012,0.398198
5,david ortiz,2013,0.395434
6,david ortiz,2014,0.338941
7,david ortiz,2015,0.371254
8,david ortiz,2016,0.397737
9,mike trout,2010,0.871429
